# SEO Notebook

In [1]:
"""A framework for conducting SEO investigations, using Google Spreadsheets for I/O."""

'A framework for conducting SEO investigations, using Google Spreadsheets for I/O.'

In [2]:
import notebook_finder    #Allows .ipynb files to be loaded like filename.py modules
import goodsheet          #Handles OAuth login for all relevant Google services
from imp import reload    #Keeps in-memory functions reflecting your pipulate.ipynb edits
import shelve, requests

In [3]:
rows_to_batch = 5
cache_html = True
google_sheet_name = 'Lookups'
google_sheet_tab  = 'Sheet1'

In [4]:
if __name__ == '__main__':
    import pipulate
    reload(pipulate)
    import pipulate #intentional
    worksheet = goodsheet.connect.open(google_sheet_name).worksheet(google_sheet_tab)
    pipulate_funcs = [x for x in dir(pipulate) if x[0] is not '_']
    func_dict = {x.lower():eval('pipulate.%s' % x) for x in pipulate_funcs}
    try:
        rows = worksheet.row_count
        cols = worksheet.col_count
        end_range = worksheet.get_addr_int(rows, cols)
        #cell_range = worksheet.range('A1:%s' % end_range)
        col_count = worksheet.col_count
    except:
        print("Cannot reach Google Sheets.")
        raise SystemExit()

    # Populate a list from Row 1
    row1_range = 'A1:%s' % worksheet.get_addr_int(1, col_count)
    cell_range = worksheet.range(row1_range)
    col_names = [x.value.lower() for x in cell_range]

    chunk_ranges = [(x+1, x+rows_to_batch) for x in list(range(rows)) if x%rows_to_batch == 0]        
    for chunk_dex, (row_start, row_end) in enumerate(chunk_ranges):
        top_left = worksheet.get_addr_int(row_start+1, 1)
        lower_right = worksheet.get_addr_int(row_end+1, cols)
        range_string = "%s:%s" % (top_left, lower_right)
        #print(chunk_dex+1, len(chunk_ranges))
        if chunk_dex+1 == len(chunk_ranges):
            range_string = "%s:%s" % (top_left, end_range)
        print("(%s) Pipulating chunk %s of %s..." % (range_string, chunk_dex+1, len(chunk_ranges)))
        cell_range = worksheet.range(range_string)
        row_dict = {}
        for cell_dex, acell in enumerate(cell_range):
            row, col, val = acell.row, acell.col, acell.value
            row_dict[col_names[col-1]] = val
            if col%col_count == 0:
                print("Row: %s" % row)
                response = None
                if 'url' in row_dict:
                    with shelve.open('urls') as urls:
                        if cache_html == True and row_dict['url'] in urls.keys():
                            response = urls[row_dict['url']]
                        else:
                            try:
                                response = requests.get(row_dict['url'])
                                urls[row_dict['url']] = response
                            except requests.exceptions.RequestException as e:
                                raise SystemExit()
                        row_dict['response'] = response
                for key, val in row_dict.items():
                    if not val:
                        if key in [x.lower() for x in dir(pipulate) if x[0] is not '_']:
                            success_code, new_text = func_dict[key](**row_dict)
                            row_start = cell_dex-cols+1
                            func_dex = col_names.index(key)
                            cell_to_update = row_start + func_dex
                            cell_range[cell_to_update].value = new_text
        try:
            worksheet.update_cells(cell_range)
            print("Range updated.")
        except:
            print("Update failed.")
    print("Pipulation complete!")


TypeError: not all arguments converted during string formatting